In [ ]:
# TODO Sync notebook and script 

In [1]:
import os
import lzma
import pickle
from tqdm import tqdm
import datajoint as dj
dj.config['database.prefix'] = os.environ.get('DJ_PREFIX', '')
from pipeline import meso, stack, treadmill, odor, mice, experiment, shared#, pupil - version of deeplabcut doesn't work with upgraded version of scipy, which was upgraded for griddata interpolation of masks in stitched space

# Define animal_id data to be ingested

In [5]:
# animal_id = 1571
# animal_id  = 14864
animal_id = 122

# Function to insert metadata

In [6]:
directory = f'{os.environ.get("INGESTION_STORAGE")}/animal_id_{animal_id}/'

def insert_data(filename, table, skip_duplicates, ignore_extra_fields, allow_direct_insert):
    with lzma.open(directory + filename, "rb") as f:
        loaded_data = pickle.load(f)
        f.close()

#     print(loaded_data)

    for i in tqdm(range(len(loaded_data))):
        table.insert1(loaded_data[i],
                      skip_duplicates=skip_duplicates,
                      ignore_extra_fields=ignore_extra_fields,
                      allow_direct_insert=allow_direct_insert)

# Mice pipeline

In [ ]:
insert_data(filename='mice.Mice.xz', 
            table=mice.Mice, 
            skip_duplicates=False, 
            ignore_extra_fields=False, 
            allow_direct_insert=False)

# Shared pipeline

In [ ]:
insert_data(filename='shared.PipelineVersion.xz', 
            table=shared.PipelineVersion, 
            skip_duplicates=True, 
            ignore_extra_fields=False, 
            allow_direct_insert=False)

# Experiment pipeline

In [ ]:
insert_data('experiment.Rig.xz', experiment.Rig, True, False, False)
insert_data('experiment.Lens.xz', experiment.Lens, True, False, False)
insert_data('experiment.Aim.xz', experiment.Aim, True, False, False)
insert_data('experiment.Software.xz', experiment.Software, True, False, False)
insert_data('experiment.Session.xz', experiment.Session, False, True, False)
insert_data('experiment.Scan.xz', experiment.Scan, False, False, False)
insert_data('experiment.Scan.EyeVideo.xz', experiment.Scan.EyeVideo, False, False, False)
insert_data('experiment.Scan.BehaviorFile.xz', experiment.Scan.BehaviorFile, False, False, False)
insert_data('experiment.Scan.Laser.xz', experiment.Scan.Laser, False, False, False)
insert_data('experiment.TreadmillSpecs.xz', experiment.TreadmillSpecs, False, False, False)

In [8]:
experiment.ExperimentalIdentifier.insert(experiment.Scan.fetch('KEY'),skip_duplicates=True)

# Meso pipeline

In [ ]:
insert_data(filename='meso.Version.xz', 
            table=meso.Version, 
            skip_duplicates=False, 
            ignore_extra_fields=False, 
            allow_direct_insert=False)

In [10]:
meso.ScanInfo.populate()

Reading header...


In [ ]:
meso.Quality.populate()

In [8]:
# insert_data('meso.CorrectionChannel.xz', meso.CorrectionChannel, False, False, False)

In [9]:
meso.RasterCorrection.populate()

In [ ]:
meso.MotionCorrection.populate()

In [ ]:
meso.SummaryImages.populate()

In [ ]:
meso.Stitch.populate()

In [ ]:
insert_data('meso.SegmentationTask.xz', meso.SegmentationTask, False, False, False)
insert_data('meso.Segmentation.xz', meso.Segmentation, False, False, True)
insert_data('meso.Segmentation.Manual.xz', meso.Segmentation.Manual, False, False, True)
insert_data('meso.Segmentation.Mask.xz', meso.Segmentation.Mask, False, False, True)

In [ ]:
meso.Fluorescence.populate()

In [ ]:
meso.MaskClassification.populate()

In [ ]:
meso.ScanSet.populate()

In [ ]:
meso.Activity.populate()

In [ ]:
meso.ScanDone.populate()

# WIP - Automated glomeruli segmentation

In [8]:
entry_segmentation_task=[]

for field in [1,2,3]:
    entry_segmentation_task.append({'animal_id': animal_id,
      'session': 1,
      'scan_idx': 1,
      'field': field,
      'channel': 1,
      'segmentation_method': 2,
      'compartment': 'unknown'})
meso.SegmentationTask.insert(entry_segmentation_task)  

In [ ]:
meso.Segmentation.populate()

In [ ]:
# (meso.SegmentationTask & 'segmentation_method=2').delete()
# (meso.Segmentation & 'segmentation_method=2').delete()

# Odor pipeline

In [ ]:
insert_data('odor.Odorant.xz', odor.Odorant, False, False, False)
insert_data('odor.OdorSolution.xz', odor.OdorSolution, False, False, False)
insert_data('odor.OdorSession.xz', odor.OdorSession, False, False, False)
insert_data('odor.OdorConfig.xz', odor.OdorConfig, False, False, False)
insert_data('odor.OdorRecording.xz', odor.OdorRecording, False, False, False)
insert_data('odor.MesoMatch.xz', odor.MesoMatch, False, False, False)

In [ ]:
odor.OdorTrials.populate()

In [ ]:
odor.OdorSync.populate()

In [ ]:
odor.Respiration.populate()

In [ ]:
odor.OdorAnalysis.populate()

# Treadmill pipeline

In [9]:
treadmill.Treadmill.populate()

# TODO: Stack pipeline

In [ ]:
experiment.Stack.insert1([121,8,1,'meso','unset','stack','scanimage','2017b',0,0,0,'','2020-12-09 11:46:48'])#unsure of depths
experiment.Stack.Filename.insert1([121,8,1,1,'121_8_00001',0])#{animal_id, session, stack_idx, filename_idx, filename, surf_depth}) #unsure of depths

In [ ]:
stack.StackInfo.populate()
stack.Quality.populate()
stack.CorrectionChannel.insert1([121,8,1,1])#correct?
stack.RasterCorrection.populate()
stack.MotionCorrection.populate()
stack.Stitching.populate()
stack.CorrectedStack.populate()
stack.PreprocessedStack.populate()#dj.conn.ping required
# stack.Surface.populate() # did not populate
stack.SegmentationTask.insert1([121,8,1,1,1,2,'soma'])#correct?

In [ ]:
stack.Segmentation.populate()
stack.RegistrationTask.insert1([121,8,1,1,,5])#correct?
stack.Registration.populate()
stack.FieldSegmentation.populate()
stack.RegistrationOverTime.populate()
stack.Drift.populate()
stack.StackSet.populate()
stack.Area.populate()